In [1]:
import pandas as pd
import numpy as np
import datasets
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from scipy.sparse import hstack
from sklearn.model_selection import RandomizedSearchCV
from tabulate import tabulate

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()
train_data_path = "./data/English dataset/train.jsonl"
test_data_path = "./data/English dataset/test.jsonl"

c:\Users\timna\Desktop\ISsem2_Cmpy\SkupnoAng\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\timna\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\timna\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\timna\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
def pretty_print_report_dict(report):
	report_df = pd.DataFrame(report).transpose()
	report_df = report_df.round(3)

	class_metrics = report_df.iloc[:-3, :].copy()

	summary_metrics = report_df.iloc[-3:, :].copy()
	summary_metrics = summary_metrics.drop(columns=['support'])

	print("CLASS PERFORMANCE")
	print(tabulate(class_metrics, headers='keys', tablefmt='heavy_outline', numalign="center"))
	print()
	print("GLOBAL AVERAGES")
	print(tabulate(summary_metrics, headers='keys', tablefmt='heavy_outline', numalign="center"))

# Data preprocessing

In [3]:
def preprocess_text(text): # From the labs
	# Tokenize the text into words
	words = word_tokenize(text.lower())  # Convert text to lowercase

	# Remove punctuation
	table = str.maketrans('', '', string.punctuation)
	words = [word.translate(table) for word in words if word.isalpha()]

	# Remove stopwords
	stop_words = set(stopwords.words('english'))
	words = [word for word in words if word not in stop_words]

	# Lemmatization
	lemmatized_words = [lemmatizer.lemmatize(word) for word in words]

	# Join the words back into a string
	preprocessed_text = ' '.join(lemmatized_words)
	return preprocessed_text

(We load the dataset. We join togeder eintailment and not mentioned, so we can focus on predicting only if something is a contradiction)

In [4]:
train_data = pd.DataFrame(datasets.load_dataset("json", data_files=train_data_path)["train"])
test_data = pd.DataFrame(datasets.load_dataset("json", data_files=test_data_path)["train"])

label_map = {"Contradiction": 1, "Entailment": 0, "NotMentioned": 0}
train_data["label"] = train_data["label"].map(label_map)
test_data["label"] = test_data["label"].map(label_map)

train_data = train_data.drop("doc_id", axis=1)
train_data = train_data.drop("key", axis=1)
test_data = test_data.drop("doc_id", axis=1)
test_data = test_data.drop("key", axis=1)

(After we load the dataset, we inspect it for class inbalance)

In [5]:
train_data["label"].value_counts(normalize=True)

label
0    0.883048
1    0.116952
Name: proportion, dtype: float64

(We can see that most of the data isn't contradictions. The data is quite imbalanced)

In [6]:
longest_premise = max(train_data['premise'].apply(len).max(), test_data['premise'].apply(len).max())
longest_hypotises = max(train_data['hypothesis'].apply(len).max(), test_data['hypothesis'].apply(len).max())
longest_sentance = max(longest_premise, longest_hypotises)
print("Longest premise: ", longest_premise)
print("Longest hypothesis: ", longest_hypotises)

print("---------------------------------")
mean = np.mean(train_data['premise'].apply(len))
std = np.std(train_data['premise'].apply(len))

print("Mean premise length: ", mean)
print("+1 std: ", mean+std)
print("+2 std: ", mean+2*std)
print("+3 std: ", mean+3*std)

Longest premise:  3098
Longest hypothesis:  162
---------------------------------
Mean premise length:  296.27826449728826
+1 std:  651.2505192635402
+2 std:  1006.2227740297922
+3 std:  1361.1950287960442


(We inspect the lenght of the data. We do this to see if it would be beneficial only keeping smaller sizes of the data, so we can cleanly feed it into BERT model. We conclude that we would need to thin our data too much to be worth it)

# Traditional ML

In [20]:
vectorizer_premise = TfidfVectorizer()
vectorizer_hypothesis = TfidfVectorizer()

train_data_vectorised = train_data.copy()

X_premise= vectorizer_premise.fit_transform(train_data["premise"])
X_hypothesis = vectorizer_hypothesis.fit_transform(train_data["hypothesis"])
train_data_vectorised = hstack([X_premise, X_hypothesis])

Y_premise = vectorizer_premise.transform(test_data["premise"])
Y_hypothesis = vectorizer_hypothesis.transform(test_data["hypothesis"])
test_data_vectorised = hstack([Y_premise, Y_hypothesis])

## Logistic regression

In [79]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

grid_serch_dict = {
	"l1_ratio": [0, 0.5, 1],
	"C": [0.1, 0.5, 1.0, 2.0, 10, 50],
	"class_weight": [None, "balanced"],
	"solver": ["saga"]
}

logreg_model = RandomizedSearchCV(LogisticRegression(max_iter=5000), grid_serch_dict, n_iter=10, cv=3, scoring='f1')
logreg_model.fit(train_data_vectorised, train_data["label"])

predictions = logreg_model.predict(test_data_vectorised)

display_params = [[k, str(v)] for k, v in logreg_model.best_params_.items()]
print(tabulate(display_params, headers=["Hyperparameter", "Value"], tablefmt="heavy_outline"))

c:\Users\timna\Desktop\ISsem2_Cmpy\SkupnoAng\.venv\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Hyperparameter   ┃ Value    ┃
┣━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━┫
┃ solver           ┃ saga     ┃
┃ l1_ratio         ┃ 0.5      ┃
┃ class_weight     ┃ balanced ┃
┃ C                ┃ 50       ┃
┗━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━┛


In [82]:
report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
pretty_print_report_dict(report_dict)

CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.982    ┃  0.974   ┃   0.978    ┃   1871    ┃
┃ 1  ┃    0.795    ┃  0.845   ┃   0.819    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.961    ┃  0.961   ┃   0.961    ┃
┃ macro avg    ┃    0.888    ┃   0.91   ┃   0.899    ┃
┃ weighted avg ┃    0.962    ┃  0.961   ┃   0.961    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

grid_serch_dict = {
	"n_estimators": [50, 100, 200],
	"max_depth": [None, 5, 10, 20],
	"min_samples_split": [2, 5, 10, 20, 50],
	"class_weight": [None, "balanced"]
}

rf_model = RandomizedSearchCV(RandomForestClassifier(random_state=67), grid_serch_dict, n_iter=15, cv=3, scoring='f1')
rf_model.fit(train_data_vectorised, train_data["label"])

predictions = rf_model.predict(test_data_vectorised)

display_params = [[k, str(v)] for k, v in rf_model.best_params_.items()]
print(tabulate(display_params, headers=["Hyperparameter", "Value"], tablefmt="heavy_outline"))

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┓
┃ Hyperparameter    ┃ Value    ┃
┣━━━━━━━━━━━━━━━━━━━╋━━━━━━━━━━┫
┃ n_estimators      ┃ 50       ┃
┃ min_samples_split ┃ 2        ┃
┃ max_depth         ┃ None     ┃
┃ class_weight      ┃ balanced ┃
┗━━━━━━━━━━━━━━━━━━━┻━━━━━━━━━━┛


In [74]:
report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
pretty_print_report_dict(report_dict)

CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.975    ┃   0.99   ┃   0.983    ┃   1871    ┃
┃ 1  ┃    0.905    ┃  0.782   ┃   0.839    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.968    ┃  0.968   ┃   0.968    ┃
┃ macro avg    ┃    0.94     ┃  0.886   ┃   0.911    ┃
┃ weighted avg ┃    0.967    ┃  0.968   ┃   0.967    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


## SVC

In [85]:
from sklearn.svm import SVC

grid_serch_dict = {
	"C": [0.1, 0.5, 1.0, 2.0, 10, 50],
	"kernel": ["linear", "sigmoid", "rbf"],
	"class_weight": [None, "balanced"]
}

svm_model = RandomizedSearchCV(SVC(), grid_serch_dict, n_iter=10, cv=3, scoring='f1')
svm_model.fit(train_data_vectorised, train_data["label"])

predictions = svm_model.predict(test_data_vectorised)

display_params = [[k, str(v)] for k, v in svm_model.best_params_.items()]
print(tabulate(display_params, headers=["Hyperparameter", "Value"], tablefmt="heavy_outline"))

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ Hyperparameter   ┃ Value   ┃
┣━━━━━━━━━━━━━━━━━━╋━━━━━━━━━┫
┃ kernel           ┃ rbf     ┃
┃ class_weight     ┃ None    ┃
┃ C                ┃ 10      ┃
┗━━━━━━━━━━━━━━━━━━┻━━━━━━━━━┛


In [86]:
report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
pretty_print_report_dict(report_dict)

CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃    0.981    ┃  0.989   ┃   0.985    ┃   1871    ┃
┃ 1  ┃    0.898    ┃  0.841   ┃   0.869    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.973    ┃  0.973   ┃   0.973    ┃
┃ macro avg    ┃    0.94     ┃  0.915   ┃   0.927    ┃
┃ weighted avg ┃    0.973    ┃  0.973   ┃   0.973    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛


# Transformer-Based Classifier

In [16]:
from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset

model_name = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
def preprocess_function(examples):
	t =  tokenizer(examples["premise"], examples["hypothesis"], truncation="only_first", stride=64, return_overflowing_tokens=True, padding='max_length', max_length=128)
	t["labels"] = examples["label"]
	return t

In [17]:
dataset = Dataset.from_pandas(train_data)
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 7191/7191 [00:18<00:00, 395.50 examples/s]


In [20]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
	output_dir="./artifacts",
	learning_rate=2e-5,
	per_device_train_batch_size=16,
	num_train_epochs=2,
	weight_decay=0.01,
	save_strategy="steps",
	save_steps=80, 
	save_total_limit=3, 
	metric_for_best_model="macro_f1"
)

trainer = Trainer(
	model=model,
	args=training_args,
	train_dataset=tokenized_dataset,
)

trainer.train(resume_from_checkpoint=False)
tokenizer.save_pretrained("./trained_model_ex3")
trainer.save_model("./trained_model_ex3")

TypeError: BertForSequenceClassification.forward() got an unexpected keyword argument 'num_items_in_batch'

In [10]:
from transformers import pipeline

#classifier = pipeline("text-classification", model="./trained_model_ex3", tokenizer="./trained_model_ex3")
from transformers import AutoTokenizer



tokenizer = AutoTokenizer.from_pretrained("kiddothe2b/longformer-mini-1024")
classifier = pipeline("text-classification", model="./artifacts/checkpoint-150", tokenizer=tokenizer)

Device set to use cpu


In [18]:
test_dataset = Dataset.from_pandas(test_data)
tokenized_dataset = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 2091/2091 [00:05<00:00, 378.25 examples/s]


In [19]:
from transformers import Trainer
from sklearn.metrics import classification_report

trainer = Trainer(model=model)  # no need for args for evaluation 
predictions_procentages = trainer.predict(tokenized_dataset)
predictions = predictions_procentages.predictions.argmax(-1)

report_dict = classification_report(test_data["label"], predictions, zero_division=0, output_dict=True)
pretty_print_report_dict(report_dict)

c:\Users\timna\Desktop\ISsem2_Cmpy\SkupnoAng\.venv\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


CLASS PERFORMANCE
┏━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃    ┃  precision  ┃  recall  ┃  f1-score  ┃  support  ┃
┣━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━╋━━━━━━━━━━━┫
┃ 0  ┃      1      ┃  0.051   ┃   0.097    ┃   1871    ┃
┃ 1  ┃    0.11     ┃    1     ┃   0.199    ┃    220    ┃
┗━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┻━━━━━━━━━━━┛

GLOBAL AVERAGES
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━┓
┃              ┃  precision  ┃  recall  ┃  f1-score  ┃
┣━━━━━━━━━━━━━━╋━━━━━━━━━━━━━╋━━━━━━━━━━╋━━━━━━━━━━━━┫
┃ accuracy     ┃    0.151    ┃  0.151   ┃   0.151    ┃
┃ macro avg    ┃    0.555    ┃  0.525   ┃   0.148    ┃
┃ weighted avg ┃    0.906    ┃  0.151   ┃   0.107    ┃
┗━━━━━━━━━━━━━━┻━━━━━━━━━━━━━┻━━━━━━━━━━┻━━━━━━━━━━━━┛
